In this post, I will go over tensorflow OpenCV object detection module. To be honest, I haven't used OpenCV for quite some time. And after recently looking into it, I have realized how awesome OpenCV has become. It <b>now has a dedicated DNN (deep neural network) module</b>. This module also has <b>functionality to load Caffe and Tensorflow trained networks.</b> I am just so happy to see that functionality in OpenCV. Just think about it, you can use your trained networks within OpenCV.

Alright, enough blabbering, let's get to the point. So in this post, I will use tensorflow detection module to load a trained tensorflow network and use this network to apply object detection to a webcam stream. So in the end, we will have a display that shows webcam stream and in the stream we modify the frames and display detected objects with rectangles. 

Alright, first thing is to get camera stream and display it. Later on we will expand this code to apply object detection to each frame instead of just displaying. 

In [1]:
#import sys

#print(sys.executable)
#print(sys.path)

#sys.path.append('/home/yutas/anaconda2/envs/testcaffe/lib/python2.7/site-packages/')

import cv2 as cv
import tensorflow as tf
import numpy as np

F:\DEVELOPMENT\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
cam = cv.VideoCapture(0)

while True:
    ret_val, img = cam.read()

    cv.imshow('my webcam', img)

    if cv.waitKey(1) == 27: 
        break  # esc to quit

cam.release()
cv.destroyAllWindows()

Now, we got the webcam stream working, next step is to integrate this with object detection. For object detection, I will use one of the networks available in Tensorflow Object detection module website : <a href='https://github.com/opencv/opencv/wiki/TensorFlow-Object-Detection-API'> Tensorflow Object Detection API</a>

In [2]:
#classes = ['person','bicycle','car','motorcycle','airplane' ,'bus','train','truck','boat' ,'traffic light','fire hydrant',
#           'stop sign','parking meter','bench','bird','cat','dog','horse','sheep','cow','elephant','bear','zebra','giraffe' ,
#           'backpack','umbrella','handbag' ,'tie','suitcase','frisbee' ,'skis','snowboard','sports ball' ,'kite',
#           'baseball bat','baseball glove','skateboard','surfboard','tennis rack','bottle','wine glass','cup','fork','knife',
#           'spoon','bowl','banana','apple' ,'sandwich','orange','broccoli','carrot','hot dog','pizza' ,'donut' ,'cake',
#           'chair' ,'couch' ,'potted plant','bed','dining table','toilet','tv','laptop','mouse','remote','keyboard',
#           'cell phone','microwave','oven','toaster','sink','refrigerator','book','clock','vase','scissors' ,'teddy bear',
#           'hair drier','toothbrush']


classes = ["background", "person", "bicycle", "car", "motorcycle",
    "airplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant",
    "unknown", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse",
    "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "unknown", "backpack",
    "umbrella", "unknown", "unknown", "handbag", "tie", "suitcase", "frisbee", "skis",
    "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard",
    "surfboard", "tennis racket", "bottle", "unknown", "wine glass", "cup", "fork", "knife",
    "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog",
    "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "unknown", "dining table",
    "unknown", "unknown", "toilet", "unknown", "tv", "laptop", "mouse", "remote", "keyboard",
    "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "unknown",
"book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush" ]




colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [3]:
cam = cv.VideoCapture(0)   

#pb  = './inceptionv2.pb'
#pbt = './inceptionv2.pbtxt'

pb  = './mobilenet.pb'
pbt = './mobilenet.pbtxt'


cvNet = cv.dnn.readNetFromTensorflow(pb,pbt)

while True:
    ret_val, img = cam.read()

    rows = img.shape[0]
    cols = img.shape[1]
    cvNet.setInput(cv.dnn.blobFromImage(img, 1.0/127.5, (300, 300), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    cvOut = cvNet.forward()

    for detection in cvOut[0,0,:,:]:
        score = float(detection[2])
        if score > 0.3:
            left = detection[3] * cols
            top = detection[4] * rows
            right = detection[5] * cols
            bottom = detection[6] * rows
            cv.rectangle(img, (int(left), int(top)), (int(right), int(bottom)), (23, 230, 210), thickness=2)

            idx = int(detection[1])
            #box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])(startX, startY, endX, endY) = box.astype("int")
            
            #print('%s, %d',str(detection[0]),idx)
            
            # draw the prediction on the frame
            label = "{}: {:.2f}%".format(classes[idx],score * 100)
            y = top - 15 if top - 15 > 15 else top + 15
            cv.putText(img, label, (int(left), int(y)),cv.FONT_HERSHEY_SIMPLEX, 0.5, colors[idx], 2)
            
    cv.imshow('my webcam', img)

    if cv.waitKey(1) == 27: 
        break  # esc to quit

cam.release()
cv.destroyAllWindows()        
    